# This training was conducted via Google Collab

### 1. Import all necessary files from google drive

In [ ]:
!rm -rf model/
!ls
!unzip '/content/drive/My Drive/model.zip' -d /content
!ls /content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2. Install all necessary requirements, like ultralytics the framework for training yolo models

In [ ]:
!pip install ultralytics
!pip install -U scikit-learn

### 3. Setup dateset structure and split data to train, test and validation data


### 4. Create yaml with all required information of dataset

### 5. Start training and wait until it is finished

In [ ]:
!rm -rf dataset/
import os
print(os.path.abspath("dataset"))
import shutil
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import yaml
import torch

def setup_yolo_dataset(image_dir, label_dir, dataset_dir):
    """ Erstellt die YOLO Dataset Struktur """
    os.makedirs(dataset_dir, exist_ok=True)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dataset_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(dataset_dir, 'labels', split), exist_ok=True)

    images = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])
    labels = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])

    img_train, img_test, lbl_train, lbl_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    img_val, img_test, lbl_val, lbl_test = train_test_split(img_test, lbl_test, test_size=0.5, random_state=42)

    def move_files(files, src_folder, dst_folder):
        for file in files:
            shutil.move(os.path.join(src_folder, file), os.path.join(dst_folder, file))

    def copy_files(files, src_folder, dst_folder):
        for file in files:
            shutil.copy(os.path.join(src_folder, file), os.path.join(dst_folder, file))

    copy_files(img_train, image_dir, os.path.join(dataset_dir, 'images', 'train'))
    copy_files(lbl_train, label_dir, os.path.join(dataset_dir, 'labels', 'train'))
    copy_files(img_val, image_dir, os.path.join(dataset_dir, 'images', 'val'))
    copy_files(lbl_val, label_dir, os.path.join(dataset_dir, 'labels', 'val'))
    copy_files(img_test, image_dir, os.path.join(dataset_dir, 'images', 'test'))
    copy_files(lbl_test, label_dir, os.path.join(dataset_dir, 'labels', 'test'))

def create_yaml_file(dataset_dir, classes):
    """ Erstellt die dataset.yaml Datei """
    print(os.path.join(dataset_dir, 'images', 'val'))
    data_yaml = {
        'train': os.path.join(dataset_dir, 'images', 'train'),
        'val': os.path.join(dataset_dir, 'images', 'val'),
        'test': os.path.join(dataset_dir, 'images', 'test'),
        'nc': len(classes),
        'names': classes
    }
    with open(os.path.join(dataset_dir, 'dataset.yaml'), 'w') as f:
        yaml.dump(data_yaml, f)

def train_yolo_model(yaml_path, epochs=50, batch_size=16):
    """ Trainiert das YOLO-Modell """
    device = torch.device("cuda")
    model = YOLO("/content/drive/My Drive/yolov5n.pt")
    model.train(data=yaml_path, epochs=epochs, batch=batch_size, imgsz=640, device=device)

def test_model(model_path, test_image):
    """ Testet das Modell mit einem Bild """
    model = YOLO(model_path)
    results = model(test_image, save=True)
    return results

if __name__ == "__main__":
    # Drive cloude
    IMAGE_DIR = "./model/images"  # Ordner mit Bildern
    LABEL_DIR = "./model/labels"  # Ordner mit YOLO-Labels (.txt)
    DATASET_DIR = "/content/dataset"  # Zielordner für YOLO Datenstruktur

    # Classes
    CLASSES = ["ball"]

    print("Setup yolo dataset")
    setup_yolo_dataset(IMAGE_DIR, LABEL_DIR, DATASET_DIR)

    print("create yaml file")
    create_yaml_file(DATASET_DIR, CLASSES)

    print("train yolo model")
    YAML_PATH = os.path.join(DATASET_DIR, 'dataset.yaml')
    train_yolo_model(YAML_PATH, epochs=50, batch_size=16)

    # TEST_IMAGE = "./train/data/0001.png"
    # test_results = test_model("runs/detect/train/weights/best.pt", TEST_IMAGE)


### 6. Zip all files you want for download

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!zip -r result.zip /content/runs/detect/train/